# Test quantum circuits

This Jupyter notebook compares the original and Pennylane version of the generaton quantum circuit ansätze to assure the equivalence of both approaches.

In [40]:
from config import CFG
import itertools
import pennylane as qml
import numpy as np
from test_qc.ansatz import Ansatz_ZZ_X_Z, Ansatz_XX_YY_ZZ_Z
from test_qc.qgates import Identity
from config import CFG

## 1. ZZ_X_Z circuit

### 1.1. Original circuit

In [27]:
class ZZ_X_Z_circuit:
    def __init__(self, size: int, layer: int, device_name="default.qubit", config=None, theta=None):
        self.size = size
        self.n_qubits = self.size
        self.layer = layer
        self.config = config or self.default_config()
        self.device = qml.device(device_name, wires=self.n_qubits)
        self.n_params = self.count_n_params()
        if theta is None:
            self.theta = np.random.rand(self.n_params)
        else:
            self.thet = theta
        self.qnode = qml.QNode(self.circuit, self.device, interface="auto")  # torch or auto

    def default_config(self):
        class CFG:
            extra_ancilla = True
            do_ancilla_1q_gates = True
            ancilla_topology = "total"  # "total", "bridge", or "ansatz"
            ancilla_connect_to = None
        return CFG()

    def count_n_params(self):
        n_params = 0
        base_size = self.size - 1 if self.config.extra_ancilla else self.size

        # 1-qubit RX and RZ
        n_params += self.size * 2

        if self.config.extra_ancilla and self.config.do_ancilla_1q_gates:
            n_params += 2  # for ancilla

        # 2-qubit ZZ between neighbors
        n_params += base_size - 1

        if self.config.extra_ancilla:
            if self.config.ancilla_topology == "total":
                n_params += base_size
            if self.config.ancilla_topology == "bridge":
                n_params += 1
            if self.config.ancilla_topology in ["bridge", "ansatz"]:
                n_params += 1

        return n_params * self.layer

    def circuit(self, theta, initial_state):
        qml.StatePrep(initial_state, wires=range(self.n_qubits))
        base_size = self.size - 1 if self.config.extra_ancilla else self.size
        ancilla_wire = base_size  # if extra ancilla is enabled

        idx = 0
        for _ in range(self.layer):
            for i in range(base_size):
                qml.RX(theta[idx], wires=i)
                idx += 1
                qml.RZ(theta[idx], wires=i)
                idx += 1

            if self.config.extra_ancilla and self.config.do_ancilla_1q_gates:
                qml.RX(theta[idx], wires=ancilla_wire-1)
                idx += 1
                qml.RZ(theta[idx], wires=ancilla_wire-1)
                idx += 1

            for i in range(base_size - 1):
                qml.IsingZZ(-theta[idx], wires=[i, i + 1])
                idx += 1

            if self.config.extra_ancilla:
                if self.config.ancilla_topology == "total":
                    for i in range(base_size):
                        qml.IsingZZ(-theta[idx], wires=[i, ancilla_wire])
                        idx += 1
                if self.config.ancilla_topology == "bridge":
                    qml.IsingZZ(-theta[idx], wires=[0, ancilla_wire])
                    idx += 1
                if self.config.ancilla_topology in ["bridge", "ansatz"]:
                    target = self.config.ancilla_connect_to or base_size - 1
                    qml.IsingZZ(-theta[idx], wires=[target, ancilla_wire])
                    idx += 1
        return qml.state()

    def run(self, initial_state, theta=None):
        theta = theta if theta is not None else self.theta
        return self.qnode(theta, initial_state)


In [28]:
def create_ghz_state_vector(num_qubits):
    # The dimension of the Hilbert space is 2^num_qubits
    dimension = 2**num_qubits
    # Initialize a zero vector of the correct dimension
    ghz_state = np.zeros(dimension, dtype=complex)
    # Set the amplitudes for the |00...0> and |11...1> states
    # The index for |00...0> is 0
    ghz_state[0] = 1 / np.sqrt(2)
    # The index for |11...1> is 2^num_qubits - 1
    ghz_state[dimension - 1] = 1 / np.sqrt(2)
    return ghz_state

In [29]:
# Define a maximally entangled 4-qubit state for testing
size = CFG.system_size
layers = CFG.gen_layers
initial_state = create_ghz_state_vector(size)

- Original

In [30]:
ansatz = Ansatz_ZZ_X_Z(config=CFG)
np.random.seed(42)
theta = np.random.rand(ansatz.n_params)
circuit = ansatz.construct_qcircuit_ZZ_X_Z(theta=theta).get_mat_rep()
state = np.asarray(circuit @ initial_state)[0]
print(state)

[ 0.11402499-0.28696683j -0.38430068+0.09898358j -0.40946981-0.1169808j
  0.10142516+0.19085783j -0.17826996-0.23588362j -0.17996919+0.09606368j
  0.07513894+0.33365511j -0.51513312-0.08790705j]


- Pennylane

In [31]:
# Define a maximally entangled 4-qubit state for testing
size = CFG.system_size
layers = CFG.gen_layers

model = ZZ_X_Z_circuit(size=size, layer=layers, config=CFG)
out_state = model.run(initial_state, theta=theta)
print(out_state)

[ 0.11402499-0.28696683j -0.38430068+0.09898358j -0.40946981-0.1169808j
  0.10142516+0.19085783j -0.17826996-0.23588362j -0.17996919+0.09606368j
  0.07513894+0.33365511j -0.51513312-0.08790705j]


In [32]:
print('Pennylane == original is', np.allclose(state, out_state))

Pennylane == original is True


### 1.2. Extended circuit

In [33]:
class ZZ_X_Z_circuit:
    def __init__(self, size: int, layer: int, device_name="default.qubit", config=None, theta=None):
        self.size = size
        self.n_qubits = 2*self.size
        self.layer = layer
        self.config = config or self.default_config()
        self.device = qml.device(device_name, wires=self.n_qubits)
        self.n_params = self.count_n_params()
        if theta is None:
            self.theta = np.random.rand(self.n_params)
        else:
            self.thet = theta
        self.qnode = qml.QNode(self.circuit, self.device, interface="auto")  # torch or auto

    def default_config(self):
        class CFG:
            extra_ancilla = True
            do_ancilla_1q_gates = True
            ancilla_topology = "total"  # "total", "bridge", or "ansatz"
            ancilla_connect_to = None
        return CFG()

    def count_n_params(self):
        n_params = 0
        base_size = self.size - 1 if self.config.extra_ancilla else self.size

        # 1-qubit RX and RZ
        n_params += self.size * 2

        if self.config.extra_ancilla and self.config.do_ancilla_1q_gates:
            n_params += 2  # for ancilla

        # 2-qubit ZZ between neighbors
        n_params += base_size - 1

        if self.config.extra_ancilla:
            if self.config.ancilla_topology == "total":
                n_params += base_size
            if self.config.ancilla_topology == "bridge":
                n_params += 1
            if self.config.ancilla_topology in ["bridge", "ansatz"]:
                n_params += 1

        return n_params * self.layer

    def circuit(self, theta, initial_state):
        qml.StatePrep(initial_state, wires=range(self.n_qubits))
        shift = self.size
        base_size = self.size - 1 if self.config.extra_ancilla else self.size

        idx = 0
        for _ in range(self.layer):
            for i in range(base_size):
                qml.RX(theta[idx], wires=shift + i)
                idx += 1
                qml.RZ(theta[idx], wires=shift + i)
                idx += 1

            if self.config.extra_ancilla and self.config.do_ancilla_1q_gates:
                qml.RX(theta[idx], wires=shift + base_size)
                idx += 1
                qml.RZ(theta[idx], wires=shift + base_size)
                idx += 1

            for i in range(base_size - 1):
                qml.IsingZZ(-theta[idx], wires=[shift + i, shift + i + 1])
                idx += 1

            if self.config.extra_ancilla:
                if self.config.ancilla_topology == "total":
                    for i in range(base_size):
                        qml.IsingZZ(-theta[idx], wires=[shift + i, shift + base_size])
                        idx += 1
                if self.config.ancilla_topology == "bridge":
                    qml.IsingZZ(-theta[idx], wires=[shift + 0, shift + base_size])
                    idx += 1
                if self.config.ancilla_topology in ["bridge", "ansatz"]:
                    target = self.config.ancilla_connect_to or base_size - 1
                    qml.IsingZZ(-theta[idx], wires=[shift + target, shift + base_size])
                    idx += 1
        return qml.state()

    def run(self, initial_state, theta=None):
        theta = theta if theta is not None else self.theta
        return self.qnode(theta, initial_state)


In [38]:
# Define a maximally entangled 4-qubit state for testing
size = CFG.system_size
layers = CFG.gen_layers
initial_state = create_ghz_state_vector(2*size)

- Original

In [42]:
ansatz = Ansatz_ZZ_X_Z(config=CFG)
np.random.seed(42)
theta = np.random.rand(ansatz.n_params)
circuit = ansatz.construct_qcircuit_ZZ_X_Z(theta=theta).get_mat_rep()
mat = np.kron(Identity(size), circuit)
state = np.asarray(mat @ initial_state)[0]
print(state)

[ 0.0685313 -0.34711976j -0.35721845+0.05507039j -0.25105373+0.02285729j
 -0.13886547+0.17681256j -0.08822043-0.19973464j -0.16476612+0.15255817j
 -0.13503137-0.01598725j -0.08754461+0.07632217j  0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.     

- Pennylane

In [36]:
# Define a maximally entangled 4-qubit state for testing
size = CFG.system_size
layers = CFG.gen_layers

model = ZZ_X_Z_circuit(size=size, layer=layers, config=CFG)
out_state = model.run(initial_state, theta=theta)
print(out_state)

[ 0.0685313 -0.34711976j -0.35721845+0.05507039j -0.25105373+0.02285729j
 -0.13886547+0.17681256j -0.08822043-0.19973464j -0.16476612+0.15255817j
 -0.13503137-0.01598725j -0.08754461+0.07632217j  0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.     

In [44]:
print('Pennylane == original is', np.allclose(state, out_state))

Pennylane == original is True


## 2. XX_YY_ZZ_Z circuit

In [19]:
class XX_YY_ZZ_circuit:
    def __init__(self, size: int, layer: int, device_name="default.qubit", config=None, theta=None):
        self.size = size
        self.n_qubits = self.size
        self.layer = layer
        self.config = config or self.default_config()
        self.device = qml.device(device_name, wires=self.n_qubits)
        self.n_params = self.count_n_params()
        if theta is None:
            self.theta = np.random.rand(self.n_params)
        else:
            self.thet = theta
        self.qnode = qml.QNode(self.circuit, self.device, interface="auto")  # torch or auto

    def default_config(self):
        class CFG:
            extra_ancilla = True
            do_ancilla_1q_gates = True
            ancilla_topology = "total"  # "total", "bridge", or "ansatz"
            ancilla_connect_to = None
        return CFG()

    def count_n_params(self):
        n_params = 0
        size = self.size
        if self.config.extra_ancilla:
            size -= 1

        entg_list = ["XX", "YY", "ZZ"]
        for _ in range(self.layer):
            # First 1 qubit gates
            for i in range(size):
                n_params += 1
            # Ancilla 1q gates for: total, bridge and disconnected:
            if self.config.extra_ancilla and self.config.do_ancilla_1q_gates:
                n_params += 1

            # Then 2 qubit gates:
            for i, gate in itertools.product(range(size - 1), entg_list):
                n_params += 1
            # Ancilla ancilla coupling (2q) logic for: total and bridge
            if self.config.extra_ancilla:
                if self.config.ancilla_topology == "total":
                    for i, gate in itertools.product(range(size), entg_list):
                        n_params += 1
                if self.config.ancilla_topology == "bridge":
                    for gate in entg_list:
                        n_params += 1
                if self.config.ancilla_topology in ["bridge", "ansatz"]:
                    qubit_to_connect_to = self.config.ancilla_connect_to if self.config.ancilla_connect_to is not None else size - 1
                    for gate in entg_list:
                        n_params += 1

        return n_params     


    def circuit(self, theta, initial_state):
        qml.StatePrep(initial_state, wires=range(self.n_qubits))
        size = self.size - 1 if self.config.extra_ancilla else self.size

        idx = 0
        entg_list = ["XX", "YY", "ZZ"]
        for _ in range(self.layer):
            # First 1 qubit gates
            for i in range(size):
                qml.RZ(theta[idx], wires=i)
                idx += 1
            # Ancilla 1q gates for: total, bridge and disconnected:
            if self.config.extra_ancilla and self.config.do_ancilla_1q_gates:
                qml.RZ(theta[idx], wires=size)
                idx += 1

            # Then 2 qubit gates:
            for i, gate in itertools.product(range(size - 1), entg_list):
                if gate == "XX":
                    qml.IsingXX(2*theta[idx], wires=[i, i+1])
                    idx += 1
                elif gate == "YY":
                    qml.IsingYY(2*theta[idx], wires=[i, i+1])
                    idx += 1
                elif gate == "ZZ":
                    qml.IsingZZ(-theta[idx], wires=[i, i+1])
                    idx += 1
            # Ancilla ancilla coupling (2q) logic for: total and bridge
            if self.config.extra_ancilla:
                if self.config.ancilla_topology == "total":
                    for i, gate in itertools.product(range(size), entg_list):
                        if gate == "XX":
                            qml.IsingXX(2*theta[idx], wires=[i, size])
                            idx += 1
                        elif gate == "YY":
                            qml.IsingYY(2*theta[idx], wires=[i, size])
                            idx += 1
                        elif gate == "ZZ":
                            qml.IsingZZ(-theta[idx], wires=[i, size])
                            idx += 1
                if self.config.ancilla_topology == "bridge":
                    for gate in entg_list:
                        if gate == "XX":
                            qml.IsingXX(2*theta[idx], wires=[0, size])
                            idx += 1
                        elif gate == "YY":
                            qml.IsingYY(2*theta[idx], wires=[0, size])
                            idx += 1
                        elif gate == "ZZ":
                            qml.IsingZZ(-theta[idx], wires=[0, size])
                            idx += 1
                if self.config.ancilla_topology in ["bridge", "ansatz"]:
                    qubit_to_connect_to = self.config.ancilla_connect_to if self.config.ancilla_connect_to is not None else size - 1
                    for gate in entg_list:
                        if gate == "XX":
                            qml.IsingXX(2*theta[idx], wires=[qubit_to_connect_to, size])
                            idx += 1
                        elif gate == "YY":
                            qml.IsingYY(2*theta[idx], wires=[qubit_to_connect_to, size])
                            idx += 1
                        elif gate == "ZZ":
                            qml.IsingZZ(-theta[idx], wires=[qubit_to_connect_to, size])
                            idx += 1
        return qml.state()

    def run(self, initial_state, theta=None):
        theta = theta if theta is not None else self.theta
        return self.qnode(theta, initial_state)


- Original

In [20]:
ansatz = Ansatz_XX_YY_ZZ_Z(config=CFG)
np.random.seed(42)
theta = np.random.rand(ansatz.n_params)
circuit = ansatz.construct_qcircuit_XX_YY_ZZ_Z(theta=theta).get_mat_rep()
state = np.asarray(circuit @ initial_state)[0]
print(state)

[-0.12152705-0.23213299j  0.25830286-0.31722023j  0.02568269-0.42378534j
  0.18436363+0.31909662j -0.06586466+0.0745767j  -0.02465748+0.36681981j
 -0.19248911-0.35116381j -0.29843596-0.23115683j]


- Pennylane

In [21]:
# Define a maximally entangled 4-qubit state for testing
size = CFG.system_size
layers = CFG.gen_layers

model = XX_YY_ZZ_circuit(size=size, layer=layers, config=CFG)
out_state = model.run(initial_state, theta=theta)
print(out_state)

[-0.12152705-0.23213299j  0.25830286-0.31722023j  0.02568269-0.42378534j
  0.18436363+0.31909662j -0.06586466+0.0745767j  -0.02465748+0.36681981j
 -0.19248911-0.35116381j -0.29843596-0.23115683j]


In [23]:
print('Pennylane == original is', np.allclose(state, out_state))

Pennylane == original is True
